# Calendar Matching
[link](https://www.algoexpert.io/questions/Calendar%20Matching)

## My Solution

In [ ]:
def calendarMatching(calendar1, dailyBounds1, calendar2, dailyBounds2, meetingDuration):
    # Write your code here.
    cal1 = newCalWithBounds(calendar1, dailyBounds1)
    cal2 = newCalWithBounds(calendar2, dailyBounds2)
    combinedCal = combineCalendars(cal1, cal2)
    # print([[m[0].t_str, m[1].t_str] for m in combinedCal])
    mergedCal = mergeCalendars(combinedCal)
    # print([[m[0].t_str, m[1].t_str] for m in mergedCal])
    return computeSpareInterval(mergedCal, meetingDuration)

def newCalWithBounds(calendar, dailyBounds):
    res = [0] * (len(calendar) + 2)
    res[0] = [T('0:00'), T(dailyBounds[0])]
    for i, c in enumerate(calendar):
        res[i + 1] = [T(c[0]), T(c[1])]
    res[len(calendar) + 1] = [T(dailyBounds[1]), T('24:00')]
    return res
    
def combineCalendars(cal1, cal2):
    res = []
    i, j = 0, 0
    while i < len(cal1) and j < len(cal2):
        if cal1[i][0] < cal2[j][0]:
            res.append(cal1[i])
            i += 1
        else:
            res.append(cal2[j])
            j += 1
    if i == len(cal1):
        left = cal2
        leftStartIdx = j
    else:
        left = cal1
        leftStartIdx = i
    for idx in range(leftStartIdx, len(left)):
        res.append(left[idx])
    return res

def mergeCalendars(calendars):
    res = []
    for idx, cal in enumerate(calendars):
        if idx == 0:
            res.append(cal)
        if cal[0] <= res[-1][1]:
            res[-1][1] = max(res[-1][1], cal[1])
        else:
            res.append(cal)
    return res

def computeSpareInterval(calendars, meetingDuration):
    res = []
    for i in range(1, len(calendars)):
        prev = calendars[i - 1]
        cur = calendars[i]
        if cur[0].t_min - prev[1].t_min >= meetingDuration:
            res.append([prev[1].t_str, cur[0].t_str])
    return res
            
class T:
    def __init__(self, t_str=None):
        self.t_str = t_str
        self.t_min = None
        self.convertStrToMin()
        
    def convertStrToMin(self):
        if self.t_str == None:
            return
        tList = list(map(int, (self.t_str.split(':'))))
        t_min = tList[0] * 60 + tList[1]
        self.t_min = t_min
        
    def __lt__(self, other):
        return self.t_min < other.t_min
    
    def __le__(self, other):
        return self.t_min <= other.t_min

## Expert Solution

In [ ]:
# O(c1 + c2) time | O(c1 + c2) space - where c1 and c2 are the respective numbers of meetings in calendar1 and calendar2
def calendarMatching(calendar1, dailyBounds1, calendar2, dailyBounds2, meetingDuration):
    updatedCalendar1 = updateCalendar(calendar1, dailyBounds1)
    updatedCalendar2 = updateCalendar(calendar2, dailyBounds2)
    mergedCalendar = mergeCalendars(updatedCalendar1, updatedCalendar2)
    flattenedCalendar = flattenCalendar(mergedCalendar)
    return getMatchingAvailabilties(flattenedCalendar, meetingDuration)

def updateCalendar(calendar, dailyBounds):
    updatedCalendar = calendar[:]
    updatedCalendar.insert(0, ["0:00", dailyBounds[0]])
    updatedCalendar.append([dailyBounds[1], "23:59"])
    return list(map(lambda m: [timeToMinutes(m[0]), timeToMinutes(m[1])], updatedCalendar))

def mergeCalendars(calendar1, calendar2):
    merged = []
    i, j = 0, 0
    while i < len(calendar1) and j < len(calendar2):
        meeting1, meeting2 = calendar1[i], calendar2[j]
        if meeting1[0] < meeting2[0]:
            merged.append(meeting1)
            i += 1
        else:
            merged.append(meeting2)
            j += 1
    while i < len(calendar1):
        merged.append(calendar1[i])
        i += 1
    while j < len(calendar2):
        merged.append(calendar2[j])
        j += 1
    return merged

def flattenCalendar(calendar):
    flattened = [calendar[0][:]]
    for i in range(1, len(calendar)):
        currentMeeting = calendar[i]
        previousMeeting = flattened[-1]
        currentStart, currentEnd = currentMeeting
        previousStart, previousEnd = previousMeeting
        if previousEnd >= currentStart:
            newPreviousMeeting = [previousStart, max(previousEnd, currentEnd)]
            flattened[-1] = newPreviousMeeting
        else:
            flattened.append(currentMeeting[:])
    return flattened

def getMatchingAvailabilties(calendar, meetingDuration):
    matchingAvailabilties = []
    for i in range(1, len(calendar)):
        start = calendar[i - 1][1]
        end = calendar[i][0]
        availabilityDuration = end - start
        if availabilityDuration >= meetingDuration:
            matchingAvailabilties.append([start, end])
    return list(map(lambda m: [minutesToTime(m[0]), minutesToTime(m[1])], matchingAvailabilties))


def timeToMinutes(time):
    hours, minutes = list(map(int, time.split(":")))
    return hours * 60 + minutes

def minutesToTime(minutes):
    hours = minutes // 60
    mins = minutes % 60
    hoursString = str(hours)
    minutesString = "0" + str(mins) if mins < 10 else str(mins)
    return hoursString + ":" + minutesString

## Thoughts